Hi everyone! Welcome to a quick walkthrough of how easy it is to form good machine learning models using SciKit Learn. In this notebook, we will use a small data pool of Medium blog posts and their associated estimated read times to see how accurately we can predict read times for unlabeled Medium posts.

# 1. Loading data into Pandas

Pandas is a data manipulation library for Python. First step in a machine learning project is typically to convert some raw data format into something we can use. Let's do so below!

In [43]:
# import statements
import pandas as pd
import numpy as np

# start with data pre-processing (i.e. convert data.txt into something we can use)

# create class to hold data points
class MediumPost():
    def __init__(self, text, minutes):
        self.text = text
        self.minutes = minutes
        
    def get_minutes(self):
        return float(self.minutes)
        
    def get_word_count(self):
        return len(self.text.split())
    
    def get_character_count(self):
        return len(self.text)
    
    def get_paragraph_count(self):
        return self.text.count("\n")

posts = []
# iterate through the raw data file and separate out data points
with open("data.txt") as raw_data:
    for line in raw_data:
        if "TEXT:\t" in line:
            text = line.split("TEXT:\t")[1]
        elif "MINUTES:\t" in line:
            minutes = line.split("MINUTES:\t")[1].strip()
            posts.append(MediumPost(text, minutes))
        else:
            text += line

# we want to store our data in a pandas dataframe
df = pd.DataFrame(columns = ["words", "characters", "paragraphs", "minutes"])

# iterate through the cleaner array of post objects and add data to our dataframe
for post in posts:
    df.loc[len(df)] = [post.get_word_count(), post.get_character_count(), post.get_paragraph_count(), post.get_minutes()]

# let's take a peak at our data
print(df.head())

    words  characters  paragraphs  minutes
0  2513.0     14134.0        33.0     10.0
1  1319.0      8519.0        32.0      6.0
2   586.0      3483.0        12.0      3.0
3   821.0      4931.0        16.0      4.0
4  1803.0     10161.0        40.0      7.0


# 2. Separating features from outputs, training data from testing data

Woah! Looking pretty good. But now one dataframe is holding onto all of our data! This is problematic for two reasons: (1) we want to predict minutes using our other features, so these columns should be separated, and (2) once we form our machine learning model we will have no data to test on, so we should separate some points to be testing points.

In [44]:
# scikit learn has a method meant to split data randomly into training and testing data... How convenient!
from sklearn.model_selection import train_test_split

outcomes = df["minutes"]
features = df.drop(["minutes"], axis=1)
# let's follow an 80/20 training/testing split
X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=.2, random_state=5)
print("Now we have %d training points" % len(X_train))
print("And %d testing points" % len(X_test))

Now we have 32 training points
And 9 testing points


# 3. GridSearch

GridSearch is an awesome algorithm to help find the best hyperparameters for a model. It works by testing every possible combination of hyperparameters you want to tune against different cross validation sets of the training data (essentially breaking down the training data into a sub training and testing set). Let's run GridSearch and return the best linear regression model.

In [45]:
# imports
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

def fit_model(X, y):
    cv_sets = ShuffleSplit(n_splits=10, test_size=0.2, random_state=43)
    regressor = LinearRegression()
    # only a few hyperparameters for lin reg: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
    params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
    scoring_fnc = make_scorer(r2_score)
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)
    grid = grid.fit(X, y)
    print("The best params from GridSearch:")
    print(grid.best_params_)
    return grid.best_estimator_

regressor = fit_model(X_train, y_train) # now we have an instance of the best regressor

The best params from GridSearch:
{'copy_X': True, 'fit_intercept': True, 'normalize': True}


# 4. Test our model on testing data

There are two important concepts to understand why there is a need to split data into training data and testing data, and those are overfitting and underfitting. Overfitting is when a model fits too much to the data set that it was trained on, then does poorly in practice because it does not generalize well. Underfitting is when a model does not fit well enough to the training data and is too general, and thus is poor on training data and testing data. Testing the model on a testing data set gives you the true insights into how well your model is performing.

In [48]:
def examine_regressor(regressor, X_test, y_test):
    y_true = []
    y_predict = []
    for i, prediction in enumerate(regressor.predict(X_test)):
        print(prediction)
        print("Predicted outcome: %.2f - Actual outcome: %.2f" % (prediction, y_test.iloc[i]))
        y_true.append(y_test.iloc[i])
        y_predict.append(prediction)
    print("Score of model: %.2f" % r2_score(y_true, y_predict))
    
examine_regressor(regressor, X_test, y_test)

5.16318277327766
Predicted outcome: 5.16 - Actual outcome: 6.00
10.33065676523207
Predicted outcome: 10.33 - Actual outcome: 11.00
14.17129954955664
Predicted outcome: 14.17 - Actual outcome: 14.00
3.73266546922257
Predicted outcome: 3.73 - Actual outcome: 4.00
10.404199121515667
Predicted outcome: 10.40 - Actual outcome: 10.00
4.787115875814991
Predicted outcome: 4.79 - Actual outcome: 5.00
7.786676694964547
Predicted outcome: 7.79 - Actual outcome: 7.00
11.426983111641174
Predicted outcome: 11.43 - Actual outcome: 11.00
7.364601951662088
Predicted outcome: 7.36 - Actual outcome: 7.00
Score of model: 0.97


# 5. Playing around with our features

Are we satisfied with this? If not, we can play around with dropping different features to see which combinations perform better or worse. This will give us an idea of which features are most important in making these predictions.

In [58]:
# play around with different features to see how well they perform
features_test = features.drop(["paragraphs", "words"], axis=1)
X_train_test, X_test_test, y_train_test, y_test_test = train_test_split(features_test, outcomes, test_size=.2, random_state=5)
regressor_test = fit_model(X_train_test, y_train_test)
examine_regressor(regressor_test, X_test_test, y_test_test)

The best params from GridSearch:
{'copy_X': True, 'fit_intercept': True, 'normalize': True}
5.258395116790003
Predicted outcome: 5.26 - Actual outcome: 6.00
10.622705748422543
Predicted outcome: 10.62 - Actual outcome: 11.00
14.99800382413974
Predicted outcome: 15.00 - Actual outcome: 14.00
3.6446345321553832
Predicted outcome: 3.64 - Actual outcome: 4.00
11.762922447702284
Predicted outcome: 11.76 - Actual outcome: 10.00
4.395844208052163
Predicted outcome: 4.40 - Actual outcome: 5.00
7.23916939486367
Predicted outcome: 7.24 - Actual outcome: 7.00
11.749863908048084
Predicted outcome: 11.75 - Actual outcome: 11.00
6.501018258621089
Predicted outcome: 6.50 - Actual outcome: 7.00
Score of model: 0.93
